[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/deployment.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239303-lesson-8-deployment)

# 部署（Deployment）

## 回顾

我们已经扩展到具备记忆的代理：

* `act` —— 让模型调用特定工具
* `observe` —— 将工具输出传回模型
* `reason` —— 让模型根据工具输出推理以决定下一步（例如调用另一个工具或直接回复）
* `persist state` —— 使用内存检查点（in-memory checkpointer）以支持在中断情况下的长时间会话

## 目标

现在，我们将介绍如何将代理实际部署到本地的 Studio 以及 `LangGraph Cloud`。

In [3]:
%%capture --no-stderr
%pip install --quiet -U langgraph_sdk langchain_core

## 概念

有一些核心概念需要了解 ——

`LangGraph` —
- Python 和 JavaScript 库
- 允许创建代理工作流

`LangGraph API` —
- 将图（graph）代码打包
- 提供任务队列以管理异步操作
- 提供持久化以维护跨交互的状态

`LangSmith Deployment`（原名 `LangGraph Cloud`）——
- LangGraph API 的托管服务
- 允许从 GitHub 仓库部署图
- 提供部署的监控与跟踪
- 每个部署通过唯一 URL 可访问

`LangSmith Studio`（原名 `LangGraph Studio`）——
- 用于 LangGraph 应用的集成开发环境（IDE）
- 使用 API 作为后端，支持对图的实时测试与探索
- 可以本地运行或云端部署（详见下文）

`LangGraph SDK` ——
- 用于以编程方式与 LangGraph 图交互的 Python 库
- 为本地或云端服务的图提供一致的接口
- 支持创建客户端、访问助手（assistants）、线程管理与运行执行

## 本地测试

## Studio

**⚠️ 注意**

自录制这些视频以来，我们已更新 Studio，使其现在可以在本地运行并通过浏览器访问。这是替代视频中所示桌面应用（Desktop App）的首选运行方式。它现在被称为 _LangSmith Studio_，而非之前的 _LangGraph Studio_。详细的设置说明请参见课程开头的“Getting Setup”指南。你可以在[这里](https://docs.langchain.com/langsmith/studio)找到 Studio 的描述，而关于本地部署的具体细节请参见[这里](https://docs.langchain.com/langsmith/quick-start-studio#local-development-server)。

要启动本地开发服务器，请在本模块的 `/studio` 目录中在终端运行以下命令：

```
langgraph dev
```

你应当看到如下输出：

```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中打开上面显示的 **Studio UI** URL。

In [1]:
if 'google.colab' in str(get_ipython()):
    raise Exception("Unfortunately LangGraph Studio is currently not supported on Google Colab")

In [2]:
from langgraph_sdk import get_client

In [3]:
# This is the URL of the local development server
URL = "http://127.0.0.1:2024"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [4]:
assistants[-3]

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'context': {},
 'metadata': {'created_by': 'system'},
 'name': 'agent',
 'created_at': '2025-11-20T11:07:21.113706+00:00',
 'updated_at': '2025-11-20T11:07:21.113706+00:00',
 'version': 1,
 'description': None}

In [5]:
# We create a thread for tracking the state of our run
thread = await client.threads.create()

现在，我们可以使用 `client.runs.stream`（参见[文档](https://docs.langchain.com/oss/python/langgraph/graph-api/#stream-and-astream)）运行我们的代理，并传入：

* `thread_id`
* `graph_id`
* `input`
* `stream_mode`

我们将在后续模块中深入讨论流式（streaming）。目前请注意，我们使用 `stream_mode="values"` 来在图的每一步流式获取完整的状态值。

状态会保存在 `chunk.data` 中。

In [6]:
from langchain_core.messages import HumanMessage

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '078a4044-cd8b-4b34-90e6-78ed4bbfc54a'}
{'content': '', 'additional_kwargs': {'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 19, 'prompt_tokens': 323, 'total_tokens': 342, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'id': '019aa123258ff1d6e2fa635e30c0f38b', 'finish_reason': 'tool_calls', 'logprobs': None}, 'type': 'ai', 'name': None, 'id': 'lc_run--abf73db1-4418-42f9-a27e-66a9fd3ff97f-0', 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': '019aa1232f67a050a0b4e6875aaf440f', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 323, 'output_tokens': 19, 'total_t

## 使用云端测试（Testing with Cloud）

我们可以通过 LangSmith 将应用部署到云端，具体步骤在[这里](https://docs.langchain.com/langsmith/deployment-quickstart#deploy-from-github-with-langgraph-cloud)。

### 在 GitHub 上创建新仓库

* 进入你的 GitHub 账户
* 点击右上角的 “+” 图标并选择 `"New repository"`
* 为你的仓库命名（例如 `langchain-academy`）

### 将你的 GitHub 仓库添加为远程仓库

* 回到你在本课程开始时克隆的 `langchain-academy` 的终端
* 将你新创建的 GitHub 仓库添加为远程仓库

```bash
git remote add origin https://github.com/your-username/your-repo-name.git
```
* 推送到该仓库

```bash
git push -u origin main
```

### 将 LangSmith 与你的 GitHub 仓库连接

* 访问 [LangSmith](https://smith.langchain.com/)
* 在左侧 LangSmith 面板点击 `deployments` 选项卡
* 点击 `+ New Deployment`
* 选择你刚创建的 GitHub 仓库（例如 `langchain-academy`）
* 将 `LangGraph API config file` 指向其中一个 `studio` 目录
* 例如，对于 module-1，指向：`module-1/studio/langgraph.json`
* 设置你的 API 密钥（例如，你可以从 `module-1/studio/.env` 文件中复制）

![Screenshot 2024-09-03 at 11.35.12 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fd61c93d48e5d0f47_deployment2.png)

### 使用你的部署

然后我们可以通过几种不同方式与我们的部署进行交互：

* 使用 SDK，如前所述
* 使用 [LangGraph Studio](https://docs.langchain.com/langsmith/deployment-quickstart#3-test-your-application-in-studio)

![Screenshot 2024-08-23 at 10.59.36 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbad4fa159a09a51d601de_deployment3.png)

要在笔记本中使用 SDK，请确保已设置 `LANGSMITH_API_KEY`！

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGSMITH_API_KEY")

In [ ]:
# Replace this with the URL of your deployed graph
URL = "https://langchain-academy-8011c561878d50b1883f7ed11b32d720.default.us.langgraph.app"
client = get_client(url=URL)

# Search all hosted graphs
assistants = await client.assistants.search()

In [37]:
# Select the agent
agent = assistants[0]

In [38]:
agent

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'created_at': '2024-08-23T17:58:02.722920+00:00',
 'updated_at': '2024-08-23T17:58:02.722920+00:00',
 'config': {},
 'metadata': {'created_by': 'system'}}

In [40]:
from langchain_core.messages import HumanMessage

# We create a thread for tracking the state of our run
thread = await client.threads.create()

# Input
input = {"messages": [HumanMessage(content="Multiply 3 by 2.")]}

# Stream
async for chunk in client.runs.stream(
        thread['thread_id'],
        "agent",
        input=input,
        stream_mode="values",
    ):
    if chunk.data and chunk.event != "metadata":
        print(chunk.data['messages'][-1])

{'content': 'Multiply 3 by 2.', 'additional_kwargs': {'example': False, 'additional_kwargs': {}, 'response_metadata': {}}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8ea04559-f7d4-4c82-89d9-c60fb0502f21', 'example': False}
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27'}, 'type': 'ai', 'name': None, 'id': 'run-b0ea5ddd-e9ba-4242-bb8c-80eb52466c76', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_EQoolxFaaSVU8HrTnCmffLk7', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '1bf558e7-79ef-4f21-bb66-acafbd04677a', 'tool_call_id': 'c